# Proposition 1

Retrieve the first and last names of employees working in either the Sales or Engineering departments.

# Functional Specification

## Query Name

Employees in Sales and Engineering Departments

## Description

This query retrieves a list of employees who are associated with either the Sales or Engineering departments. It returns the first and last names of employees along with the department name, using a UNION operation to combine results from both departments.

## Inputs

- **FirstName**: The first name of the employee.
- **LastName**: The last name of the employee.
- **DepartmentName**: The name of the department where the employee works.

## Outputs

- **FirstName**: The first name of the employee.
- **LastName**: The last name of the employee.
- **DepartmentName**: The name of the department, which will be either 'Sales' or 'Engineering'.

## Steps

1. Retrieve employee details for the Sales department:
    - Join the `HumanResources.Employee` table with the `Person.Person` table using `BusinessEntityID`.
    - Join the result with the `HumanResources.EmployeeDepartmentHistory` table to get department history.
    - Further join with the `HumanResources.Department` table to obtain the department name.
    - Filter results to include only those where the department name is 'Sales'.
2. Retrieve employee details for the Engineering department following the same process as above but filter for 'Engineering'.
3. Use a UNION operation to combine the results from the Sales and Engineering queries.
4. Return the first name, last name, and department name for employees in either department.

## Assumptions

- Employee records are available in the `HumanResources.Employee` and `Person.Person` tables.
- The `BusinessEntityID` is unique across these tables.
- An employee can be associated with multiple departments in the history but only the relevant departments (Sales or Engineering) are considered in this query.

## Example Output

| FirstName | LastName | DepartmentName |
| --- | --- | --- |
| John | Doe | Sales |
| Jane | Smith | Engineering |
| Mark | Johnson | Sales |
| Emily | Davis | Engineering |

In [ ]:
    SELECT FirstName, LastName, Name as DepartmentName
    FROM HumanResources.Employee AS e
        JOIN Person.Person as p on p.BusinessEntityID = e.BusinessEntityID
        JOIN HumanResources.EmployeeDepartmentHistory AS dh ON e.BusinessEntityID = dh.BusinessEntityID
        JOIN HumanResources.Department as d ON dh.DepartmentID = d.DepartmentID
    WHERE d.Name = 'Sales'

UNION

    SELECT FirstName, LastName, Name as DepartmentName
    FROM HumanResources.Employee AS e
        JOIN Person.Person as p on p.BusinessEntityID = e.BusinessEntityID
        JOIN HumanResources.EmployeeDepartmentHistory AS dh ON e.BusinessEntityID = dh.BusinessEntityID
        JOIN HumanResources.Department as d ON dh.DepartmentID = d.DepartmentID
    WHERE d.Name = 'Engineering';


# Proposition 2

Retrieve the list of customers who have only made offline purchases and no online purchases.

# Functional Specification

## Query Name
Customers with Only Offline Orders

## Description
This query identifies customers who have placed orders exclusively offline (i.e., not online). It does this by selecting customers with orders flagged as offline and excluding those who have any orders flagged as online.

## Inputs
- **CustomerID**: The unique identifier for each customer.
- **OnlineOrderFlag**: A flag indicating whether the order was placed online (`1` for online, `0` for offline).

## Outputs
- **CustomerID**: The unique identifier of the customer who has made only offline orders.

## Steps
1. Retrieve the list of `CustomerID`s from the `Sales.SalesOrderHeader` table where `OnlineOrderFlag` is `0` (offline orders).
2. Retrieve the list of `CustomerID`s from the same table where `OnlineOrderFlag` is `1` (online orders).
3. Use the `EXCEPT` operation to exclude customers who appear in the online orders list, resulting in a list of customers who have made only offline purchases.

## Assumptions
- The data is available in the `Sales.SalesOrderHeader` table.
- The `CustomerID` is unique for each customer in the context of their orders.
- A customer can have multiple orders, some online and some offline, but this query returns only those customers with no online orders at all.

## Example Output

| CustomerID |
|------------|
| 1001       |
| 1003       |
| 1025       |
| 1030       |


In [ ]:
    SELECT CustomerID
    FROM Sales.SalesOrderHeader
    WHERE OnlineOrderFlag = 0


EXCEPT

    SELECT CustomerID
    FROM Sales.SalesOrderHeader
    WHERE OnlineOrderFlag = 1

# Proposition 3

Retrieve the list of products that have been sold in both online and offline orders.

# Functional Specification

## Query Name
Products Sold in Both Online and Offline Orders

## Description
This query identifies products that appear in sales orders associated with both online and offline purchases. It uses the `INTERSECT` operation to find common products that have been sold through both sales channels by matching the products in sales order details for orders flagged as online (`OnlineOrderFlag = 1`) and offline (`OnlineOrderFlag = 0`).

## Inputs
- **ProductID**: The unique identifier for each product.
- **Name**: The name of the product.
- **SalesOrderID**: The unique identifier for each sales order.
- **OnlineOrderFlag**: A flag indicating whether the order was placed online (`1` for online, `0` for offline).

## Outputs
- **ProductID**: The unique identifier of the product that has been sold in both online and offline orders.
- **Name**: The name of the product.

## Steps
1. Retrieve the `ProductID` and `Name` from the `Sales.SalesOrderDetail` and `Production.Product` tables for products included in online orders.
   - Join the `Sales.SalesOrderDetail` table with the `Production.Product` table using `ProductID`.
   - Filter the results by selecting only the sales orders where the `OnlineOrderFlag` is `1`.
2. Retrieve the `ProductID` and `Name` for products included in offline orders following the same approach, but filter by selecting sales orders where the `OnlineOrderFlag` is `0`.
3. Use the `INTERSECT` operation to find products common to both sets of results.
4. Return the `ProductID` and `Name` for products sold in both online and offline orders.

## Assumptions
- The data is available in the `Sales.SalesOrderDetail`, `Production.Product`, and `Sales.SalesOrderHeader` tables.
- Each product may appear in multiple sales orders, including both online and offline sales.
- The `INTERSECT` operation will only include products that exist in both sets of filtered results.

## Example Output

| ProductID | Name                |
|-----------|---------------------|
| 201       | Multi-Purpose Tool  |
| 305       | Deluxe Widget       |
| 409       | Standard Component  |
| 512       | High-Performance Part|


In [ ]:
    SELECT s.ProductID, Name
    FROM Sales.SalesOrderDetail s
        JOIN Production.Product P on p.ProductID = s.ProductID
    WHERE SalesOrderID IN (
    SELECT SalesOrderID
    FROM Sales.SalesOrderHeader
    WHERE OnlineOrderFlag = 1
)

INTERSECT

    SELECT s.ProductID, Name
    FROM Sales.SalesOrderDetail s
        JOIN Production.Product P on p.ProductID = s.ProductID
    WHERE SalesOrderID IN (
    SELECT SalesOrderID
    FROM Sales.SalesOrderHeader
    WHERE OnlineOrderFlag = 0
);


# Proposition 4

Retrieve a unified list of product category names from both product categories and subcategories.

# Functional Specification

## Query Name
Unified Product Category List

## Description
This query combines the names of product categories and product subcategories into a single list. It utilizes the `UNION` operation to ensure that only distinct category names are returned from both the `Production.ProductCategory` and `Production.ProductSubcategory` tables.

## Inputs
- **Name**: The name of the product category or subcategory.

## Outputs
- **CategoryName**: The distinct names of product categories and subcategories.

## Steps
1. Retrieve the `Name` from the `Production.ProductCategory` table and rename it as `CategoryName`.
2. Retrieve the `Name` from the `Production.ProductSubcategory` table and rename it as `CategoryName`.
3. Use the `UNION` operation to combine the results from both queries, ensuring that duplicate category names are eliminated from the final output.
4. Return the distinct `CategoryName` for each entry.

## Assumptions
- The data is available in the `Production.ProductCategory` and `Production.ProductSubcategory` tables.
- The `Name` field in both tables may contain overlapping names, which will be filtered out by the `UNION` operation to ensure uniqueness.

## Example Output

| CategoryName         |
|----------------------|
| Electronics          |
| Home & Garden        |
| Clothing             |
| Tools                |
| Toys                 |
| Office Supplies      |


In [ ]:
    SELECT Name AS CategoryName
    FROM Production.ProductCategory

UNION

    SELECT Name AS CategoryName
    FROM Production.ProductSubcategory;


# Proposition 5

Retrieve a unified list of sales orders, indicating whether they are reseller orders (with discounts) or online orders.

# Functional Specification

## Query Name
Sales Orders by Type

## Description
This query generates a list of sales orders, categorizing them as either "Reseller" or "Online". It combines reseller orders that have a unit price discount with online orders, ensuring that both types of orders are represented in the final result set.

## Inputs
- **SalesOrderID**: The unique identifier for each sales order.
- **CustomerID**: The unique identifier for each customer.
- **UnitPriceDiscount**: The discount applied to the unit price of products in the sales order.
- **OnlineOrderFlag**: A flag indicating whether the order was placed online (`1` for online, `0` for offline).

## Outputs
- **SalesOrderID**: The unique identifier for the sales order.
- **CustomerID**: The unique identifier of the customer associated with the sales order.
- **OrderType**: A string indicating the type of order (`'Reseller'` for orders with discounts, `'Online'` for online orders).

## Steps
1. Retrieve `SalesOrderID` and `CustomerID` from the `Sales.SalesOrderHeader` table where the `SalesOrderID` corresponds to orders in the `Sales.SalesOrderDetail` table with a `UnitPriceDiscount` greater than `0`. Assign the static string `'Reseller'` to the `OrderType`.
2. Retrieve `SalesOrderID` and `CustomerID` from the `Sales.SalesOrderHeader` table where the `OnlineOrderFlag` is `1`. Assign the static string `'Online'` to the `OrderType`.
3. Use the `UNION` operation to combine the results of both queries, ensuring that duplicate orders are eliminated from the final output.
4. Return the combined list of `SalesOrderID`, `CustomerID`, and `OrderType`.

## Assumptions
- The data is available in the `Sales.SalesOrderHeader` and `Sales.SalesOrderDetail` tables.
- The `SalesOrderID` is unique within the `Sales.SalesOrderHeader` table.
- The query assumes that reseller orders can be identified through the presence of a discount in the associated order details.

## Example Output

| SalesOrderID | CustomerID | OrderType |
|--------------|------------|-----------|
| 1001         | C001       | Reseller  |
| 1002         | C002       | Online    |
| 1003         | C003       | Reseller  |
| 1004         | C004       | Online    |
| 1005         | C005       | Reseller  |


In [ ]:
    SELECT SalesOrderID, CustomerID, 'Reseller' AS OrderType
    FROM Sales.SalesOrderHeader
    WHERE SalesOrderID IN (
    SELECT SalesOrderID
    FROM Sales.SalesOrderDetail
    WHERE UnitPriceDiscount > 0
)

UNION

    SELECT SalesOrderID, CustomerID, 'Online' AS OrderType
    FROM Sales.SalesOrderHeader
    WHERE OnlineOrderFlag = 1;


# Proposition 6

Retrieve a list of products that are not included in any special offers.

# Functional Specification

## Query Name

Products Not in Special Offers

## Description

This query identifies products from the `Production.Product` table that are not associated with any special offers in the `Sales.SpecialOfferProduct` table. It uses the `EXCEPT` operation to filter out products that appear in the special offers.

## Inputs

- **ProductID**: The unique identifier for each product.

## Outputs

- **ProductID**: The unique identifier of the products that are not part of any special offers.

## Steps

1. Retrieve the list of `ProductID`s from the `Production.Product` table.
2. Retrieve the list of `ProductID`s from the `Sales.SpecialOfferProduct` table.
3. Use the `EXCEPT` operation to find products in the `Production.Product` table that are not present in the `Sales.SpecialOfferProduct` table.
4. Return the `ProductID`s of the products that are not part of any special offers.

## Assumptions

- The data is available in the `Production.Product` and `Sales.SpecialOfferProduct` tables.
- Each `ProductID` is unique within the `Production.Product` table and can appear multiple times in the `Sales.SpecialOfferProduct` table if associated with multiple special offers.

## Example Output

| ProductID |
| --- |
| 101 |
| 102 |
| 103 |
| 104 |
| 105 |

In [ ]:
    SELECT ProductID
    FROM Production.Product

EXCEPT

    SELECT ProductID
    FROM Sales.SpecialOfferProduct;


# Proposition 7

Retrieve a list of products that are available in inventory but are not associated with any vendors.

# Functional Specification

## Query Name
Products in Inventory Not Linked to Vendors

## Description
This query identifies products that are present in the `Production.ProductInventory` table but do not have any associated vendors in the `Purchasing.ProductVendor` table. It uses the `EXCEPT` operation to filter out products that appear in the vendor associations.

## Inputs
- **ProductID**: The unique identifier for each product.
- **Name**: The name of the product.

## Outputs
- **ProductID**: The unique identifier of the products available in inventory but not linked to any vendor.
- **Name**: The name of the products available in inventory but not linked to any vendor.

## Steps
1. Retrieve the `ProductID` and `Name` from the `Production.ProductInventory` table, joining it with the `Production.Product` table on `ProductID`.
2. Retrieve the `ProductID` and `Name` from the `Production.Product` table, joining it with the `Purchasing.ProductVendor` table on `ProductID`.
3. Use the `EXCEPT` operation to find products in the first result set (inventory) that are not present in the second result set (vendors).
4. Return the `ProductID` and `Name` of the products that are available in inventory but are not linked to any vendors.

## Assumptions
- The data is available in the `Production.Product`, `Production.ProductInventory`, and `Purchasing.ProductVendor` tables.
- Each `ProductID` is unique within the `Production.Product` table.
- Products may exist in the inventory without having any associated vendors.

## Example Output

| ProductID | Name                |
|-----------|---------------------|
| 201       | Advanced Widget     |
| 202       | Premium Gadget      |
| 203       | Eco-Friendly Product |
| 204       | Industrial Tool     |


In [ ]:
    SELECT pi.ProductID, p.name
    FROM Production.ProductInventory AS pi
        JOIN Production.Product AS p on p.ProductID = pi.ProductID

EXCEPT

    SELECT p.ProductID, p.Name
    FROM Production.Product AS p
        JOIN Purchasing.ProductVendor AS pv ON p.ProductID = pv.ProductID


# Proposition 8

Retrieve a list of products that have been sold in online orders and are associated with special offers.

# Functional Specification

## Query Name

Products Sold Online with Special Offers

## Description

This query identifies products that have been sold through online orders and are also included in special offers. It uses the `INTERSECT` operation to find products that appear in both sets of criteria.

## Inputs

- **ProductID**: The unique identifier for each product.
- **Name**: The name of the product.
- **OnlineOrderFlag**: A flag indicating whether the order was placed online (`1` for online, `0` for offline).

## Outputs

- **ProductID**: The unique identifier of the products that meet both criteria.
- **Name**: The name of the products that meet both criteria.

## Steps

1. Retrieve the `ProductID` and `Name` from the `Production.Product` table for products sold in online orders:
    - Join the `Production.Product` table with the `Sales.SalesOrderDetail` table on `ProductID`.
    - Join the `Sales.SalesOrderDetail` table with the `Sales.SalesOrderHeader` table on `SalesOrderID`.
    - Filter the results where `OnlineOrderFlag` is `1`.
2. Retrieve the `ProductID` and `Name` from the `Production.Product` table for products that are part of special offers:
    - Join the `Production.Product` table with the `Sales.SpecialOfferProduct` table on `ProductID`.
3. Use the `INTERSECT` operation to find products common to both result sets.
4. Return the `ProductID` and `Name` of the products that have been sold in online orders and are associated with special offers.

## Assumptions

- The data is available in the `Production.Product`, `Sales.SalesOrderDetail`, `Sales.SalesOrderHeader`, and `Sales.SpecialOfferProduct` tables.
- Each `ProductID` is unique within the `Production.Product` table and can appear in multiple sales orders or special offers.

## Example Output

| ProductID | Name |
| --- | --- |
| 301 | High-End Laptop |
| 302 | Premium Headphones |
| 303 | Smart Home Device |
| 304 | Gaming Console |

In [ ]:


    SELECT p.ProductID, p.Name
    FROM Production.Product AS p
        JOIN Sales.SalesOrderDetail AS sod ON p.ProductID = sod.ProductID
        JOIN Sales.SalesOrderHeader AS soh ON sod.SalesOrderID = soh.SalesOrderID
    WHERE soh.OnlineOrderFlag = 1

INTERSECT

    SELECT p.ProductID, p.Name
    FROM Production.Product AS p
        JOIN Sales.SpecialOfferProduct AS sop ON p.ProductID = sop.ProductID;


# Proposition 9

Retrieve a list of customers who have purchased both high-priced and low-priced products.

# Functional Specification

## Query Name
Customers with High and Low-Priced Purchases

## Description
This query identifies customers who have made purchases of products that fall into two distinct price categories: those with a list price greater than 2500 and those with a list price less than 25. It uses the `INTERSECT` operation to find customers who meet both criteria.

## Inputs
- **CustomerID**: The unique identifier for each customer.
- **ListPrice**: The price at which the product is listed for sale.

## Outputs
- **CustomerID**: The unique identifier of customers who have purchased both high-priced and low-priced products.

## Steps
1. Retrieve `CustomerID` from the `Sales.SalesOrderHeader` table for customers who have purchased products with a `ListPrice` greater than 2500:
   - Join the `Sales.SalesOrderHeader` table with the `Sales.SalesOrderDetail` table on `SalesOrderID`.
   - Join the `Sales.SalesOrderDetail` table with the `Production.Product` table on `ProductID`.
   - Filter the results where `ListPrice` is greater than 2500.
2. Retrieve `CustomerID` from the `Sales.SalesOrderHeader` table for customers who have purchased products with a `ListPrice` less than 25:
   - Repeat the same joins as above but filter for `ListPrice` less than 25.
3. Use the `INTERSECT` operation to find customers who appear in both result sets.
4. Return the `CustomerID` of customers who have purchased both high-priced and low-priced products.

## Assumptions
- The data is available in the `Sales.SalesOrderHeader`, `Sales.SalesOrderDetail`, and `Production.Product` tables.
- Each `CustomerID` is unique within the `Sales.SalesOrderHeader` table.

## Example Output

| CustomerID |
|------------|
| C001       |
| C002       |
| C003       |


In [ ]:
    SELECT soh.CustomerID
    FROM Sales.SalesOrderHeader AS soh
        JOIN Sales.SalesOrderDetail AS sod ON soh.SalesOrderID = sod.SalesOrderID
        JOIN Production.Product AS p ON sod.ProductID = p.ProductID
    WHERE p.ListPrice > 2500

INTERSECT

    SELECT soh.CustomerID
    FROM Sales.SalesOrderHeader AS soh
        JOIN Sales.SalesOrderDetail AS sod ON soh.SalesOrderID = sod.SalesOrderID
        JOIN Production.Product AS p ON sod.ProductID = p.ProductID
    WHERE p.ListPrice < 25;


# Proposition 10

Retrieve a list of customers who have placed orders in both January and December.

# Functional Specification

## Query Name
Customers with Orders in January and December

## Description
This query identifies customers who have made purchases in both January and December. It uses the `INTERSECT` operation to find customers who placed orders in both specified months.

## Inputs
- **CustomerID**: The unique identifier for each customer.
- **OrderDate**: The date on which the order was placed.

## Outputs
- **CustomerID**: The unique identifier of customers who have placed orders in both January and December.

## Steps
1. Retrieve `CustomerID` from the `Sales.SalesOrderHeader` table for customers who placed orders in January:
   - Filter results where the `MONTH(OrderDate)` equals `1`.
2. Retrieve `CustomerID` from the `Sales.SalesOrderHeader` table for customers who placed orders in December:
   - Filter results where the `MONTH(OrderDate)` equals `12`.
3. Use the `INTERSECT` operation to find customers who appear in both result sets.
4. Return the `CustomerID` of customers who have placed orders in both January and December.

## Assumptions
- The data is available in the `Sales.SalesOrderHeader` table.
- Each `CustomerID` is unique within the `Sales.SalesOrderHeader` table.
- The `OrderDate` is in a valid date format.

## Example Output

| CustomerID |
|------------|
| C001       |
| C002       |
| C003       |


In [ ]:
    SELECT CustomerID
    FROM Sales.SalesOrderHeader
    WHERE MONTH(OrderDate) IN (1)

INTERSECT

    SELECT CustomerID
    FROM Sales.SalesOrderHeader
    WHERE MONTH(OrderDate) IN (12);
